### 1. Initialisation

In [37]:
#r "nuget: Microsoft.DotNet.Interactive, *-*"
#r "nuget: Microsoft.DotNet.Interactive.CSharp, *-*"
#r "nuget: Microsoft.DotNet.Interactive.Documents, *-*"


#r "nuget: Microsoft.SemanticKernel, 1.13.0"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, 1.13.0-preview"

Installed Packages Microsoft.DotNet.Interactive, 1.0.0-beta.24229.4 Microsoft.DotNet.Interactive.CSharp, 1.0.0-beta.24229.4 microsoft.dotnet.interactive.documents, 1.0.0-beta.24229.4 Microsoft.SemanticKernel, 1.13.0 Microsoft.SemanticKernel.Planners.OpenAI, 1.13.0-preview

- **Imports des espaces de noms**

In [38]:
  using Microsoft.DotNet.Interactive;
  using Microsoft.SemanticKernel;
  using Microsoft.SemanticKernel.Planning;
  using Microsoft.SemanticKernel.Connectors.OpenAI;
  using System;
  using System.IO;
  using System.Threading.Tasks;

  using SKernel = Microsoft.SemanticKernel.Kernel;
  using IKernel = Microsoft.DotNet.Interactive.Kernel;

- **Configurez l'authentification des services semantic-kernel**

Créer au besoin le fichier config/settings.json pour la config semantic-kernel

In [39]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs 

Initialisation semantic-kernel:**

In [40]:
// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

  var builder = SKernel.CreateBuilder();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);



var semanticKernel = builder.Build();

Console.WriteLine("Kernel and Semantic Kernel initialized.");

Kernel and Semantic Kernel initialized.


### 2. Mode de Fourniture des Informations

In [41]:
public enum InformationMode
{
    Variable,
    Prompt,
    File
}

var mode = InformationMode.Variable; // Changez cette valeur pour tester les différents modes

#### Mode 1: Personnalisation via une variable textuelle

In [42]:
string taskDescription = "Créer un notebook permettant de requêter DBPedia en utilisant RDF.Net";

if (mode == InformationMode.Variable)
{
    Console.WriteLine("Utilisation de la variable pour la description de la tâche.");
    var personalizedTask = taskDescription;
    Console.WriteLine(personalizedTask);
}

Utilisation de la variable pour la description de la tâche.
Créer un notebook permettant de requêter DBPedia en utilisant RDF.Net


#### Mode 2: Série de questions via le prompt de .NET Interactive

In [43]:
if (mode == InformationMode.Prompt)
{
    var questions = new[]
    {
        "Bonjour! Veuillez fournir une brève description de la tâche à accomplir.",
        "Quels sont les principaux objectifs de cette tâche?",
        "Y a-t-il des contraintes ou des conditions spécifiques à prendre en compte?",
        "Des informations supplémentaires que vous souhaitez ajouter?"
    };

    taskDescription = string.Empty;
    foreach (var question in questions)
    {
        var response = await IKernel.GetInputAsync(question);
        taskDescription += $"{question}\\n{response}\\n\\n";
    }

    Console.WriteLine("Informations recueillies :\\n" + taskDescription);
}

### 3. Personnalisation et Exécution du Notebook de Travail

In [44]:
var notebookTemplatePath = "./Workbook-Template.ipynb";
var notebookPath = @$"./Workbook-{DateTime.Now.Date.ToString("yyyy-MM-dd")}.ipynb";
var notebookOutputPath = @$"./Workbook-{DateTime.Now.Date.ToString("yyyy-MM-dd")}-output.ipynb";

string notebookContent = File.ReadAllText(notebookTemplatePath);

notebookContent = notebookContent.Replace("{{TASK_DESCRIPTION}}", taskDescription);


File.WriteAllText(personalizedNotebookPath, notebookContent);
Console.WriteLine("Notebook personnalisé prêt à l'exécution.");

Notebook personnalisé prêt à l'exécution.


### 4. Boucle Récurrente pour l'Exécution du Notebook

- **Classe en charge de charger, d'exécuter et renvoyer un le résultat d'un notebook**

In [45]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Documents;
using Microsoft.DotNet.Interactive.Commands;
using Microsoft.DotNet.Interactive.Events;
using System.Reactive.Linq;
using System.Text;
using System.Threading;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;



public class NotebookExecutor
{

    public static string PlainTextValue(DisplayEvent @event)
    {
        return @event.FormattedValues.FirstOrDefault()?.Value ?? string.Empty;
    }

    private readonly CompositeKernel _kernel;

    public NotebookExecutor(CompositeKernel kernel)
    {
        _kernel = kernel;
    }

    public async Task<InteractiveDocument> RunNotebookAsync(
        InteractiveDocument notebook,
        IDictionary<string, string>? parameters = null,
        CancellationToken cancellationToken = default)
    {
        var resultDocument = new InteractiveDocument();

        if (parameters is not null)
        {
            parameters = new Dictionary<string, string>(parameters, StringComparer.InvariantCultureIgnoreCase);
            // var inputKernel = _kernel.ChildKernels.OfType<InputKernel>().FirstOrDefault();
            // if (inputKernel is not null)
            // {
            //     inputKernel.GetInputValueAsync = key =>
            //     {
            //         if (parameters.TryGetValue(key, out var value))
            //         {
            //             return Task.FromResult<string?>(value);
            //         }
            //         else
            //         {
            //             return Task.FromResult<string?>(null);
            //         }
            //     };
            // }
        }

        foreach (var element in notebook.Elements)
        {
            var command = new SubmitCode(element.Contents, element.KernelName);
            var events = _kernel.KernelEvents.Replay();
            using var connect = events.Connect();

            var startTime = DateTimeOffset.Now;
            var result = _kernel.SendAsync(command, cancellationToken);
            var tcs = new TaskCompletionSource();
            StringBuilder? stdOut = default;
            StringBuilder? stdErr = default;
            var outputs = new List<InteractiveDocumentOutputElement>();

            using var _ = events.Subscribe(@event =>
            {
                switch (@event)
                {
                    case IncompleteCodeSubmissionReceived incomplete when incomplete.Command == command:
                        break;
                    case CompleteCodeSubmissionReceived complete when complete.Command == command:
                        break;
                    case ErrorProduced errorProduced:
                        outputs.Add(CreateErrorOutputElement(errorProduced));
                        break;
                    case StandardOutputValueProduced standardOutputValueProduced:
                        stdOut ??= new StringBuilder();
                        stdOut.Append(PlainTextValue(standardOutputValueProduced));
                        break;
                    case StandardErrorValueProduced standardErrorValueProduced:
                        stdErr ??= new StringBuilder();
                        stdErr.Append(PlainTextValue(standardErrorValueProduced));
                        break;
                    case DisplayedValueProduced displayedValueProduced:
                        outputs.Add(CreateDisplayOutputElement(displayedValueProduced));
                        break;
                    case DisplayedValueUpdated displayedValueUpdated:
                        outputs.Add(CreateDisplayOutputElement(displayedValueUpdated));
                        break;
                    case ReturnValueProduced returnValueProduced:
                        if (returnValueProduced.Value is DisplayedValue)
                        {
                            break;
                        }
                        outputs.Add(CreateDisplayOutputElement(returnValueProduced));
                        break;
                    case CommandFailed failed when failed.Command == command:
                        if (CreateBufferedStandardOutAndErrElement(stdOut, stdErr) is { } te)
                        {
                            outputs.Add(te);
                        }
                        outputs.Add(CreateErrorOutputElement(failed));
                        tcs.SetResult();
                        break;
                    case CommandSucceeded succeeded when succeeded.Command == command:
                        if (CreateBufferedStandardOutAndErrElement(stdOut, stdErr) is { } textElement)
                        {
                            outputs.Add(textElement);
                        }
                        tcs.SetResult();
                        break;
                }
            });

            await tcs.Task;
            var resultElement = new InteractiveDocumentElement(element.Contents, element.KernelName, outputs.ToArray())
            {
                Metadata = new Dictionary<string, object>
                {
                    { "dotnet_repl_cellExecutionStartTime", startTime },
                    { "dotnet_repl_cellExecutionEndTime", DateTimeOffset.Now }
                }
            };

            resultDocument.Add(resultElement);
        }

        var defaultKernelName = _kernel.DefaultKernelName;
        var defaultKernel = _kernel.ChildKernels.SingleOrDefault(k => k.Name == defaultKernelName);
        var languageName = defaultKernel?.KernelInfo.LanguageName ?? notebook.GetDefaultKernelName() ?? "C#";

        resultDocument.Metadata["kernelspec"] = new Dictionary<string, object>
        {
            { "name", defaultKernel?.Name ?? "csharp" },
            { "language", languageName }
        };
        return resultDocument;
    }

    private TextElement? CreateBufferedStandardOutAndErrElement(StringBuilder? stdOut, StringBuilder? stdErr)
    {
        if (stdOut is null && stdErr is null)
        {
            return null;
        }

        var sb = new StringBuilder();
        if (stdOut is { })
        {
            sb.Append(stdOut);
        }

        if (stdOut is { } && stdErr is { })
        {
            sb.Append("\n\n");
        }

        if (stdErr is { })
        {
            sb.Append(stdErr);
        }

        return new TextElement(sb.ToString(), "stdout");
    }

    private DisplayElement CreateDisplayOutputElement(DisplayEvent displayedValueProduced) =>
        new(displayedValueProduced.FormattedValues.ToDictionary(v => v.MimeType, v => (object)v.Value));

    private ErrorElement CreateErrorOutputElement(ErrorProduced errorProduced) =>
        new("Error", errorProduced.Message);

    private ErrorElement CreateErrorOutputElement(CommandFailed failed) =>
        new("Error", failed.Message, failed.Exception?.StackTrace?.Split('\n') ?? Array.Empty<string>());
}


- **Analyse et Proposition de Mise à Jour:**

In [46]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Planning;
using Microsoft.DotNet.Interactive.Documents;
using Microsoft.DotNet.Interactive.Documents.Jupyter;
using System.Threading.Tasks;
using System.IO;

// public class WorkbookInteraction
// {
//     [KernelFunction]
//     [Description("Returns an updated version of the workbook after pondering the task and the outputs of the current version.")]
//     public string UpdateWorkbook([Description("the new version of the workbook in ipynb json format")] string updatedWorkbook)
//     {
        
        
//     }

//     [KernelFunction]
//     [Description("Returns the final answer if last version of the workbook allowed to conclude the task.")]
//     public string SendFinalAnswer([Description("The final answer to the task at hand")] string answer)
//     {
//         var resultDisplay = display("Final Answer: " + answer);
//         return answer;
//     }
// }

// var chatHistory = new ChatHistory();
// var resultDisplay = display("Collecte d'informations en cours...");

// foreach (var question in questions)
// {
//     chatHistory.AddUserMessage(question);
//     var result = await planner.ExecuteAsync(kernel, question, chatHistory);
//     chatHistory.AddAssistantMessage(result.FinalAnswer);
//     chatHistory.AddUserMessage(result.FinalAnswer); // Store the user's answer
//     resultDisplay.Update(resultDisplay.GetValue<string>() + "\n\n" + "Question: " + question + "\nAnswer: " + result.FinalAnswer);
// }

#pragma warning disable SKEXP0060

public class NotebookUpdater
{
    private readonly NotebookExecutor _executor;
    private readonly FunctionCallingStepwisePlanner _planner;
    private readonly SKernel _semanticKernel;

    public NotebookUpdater(NotebookExecutor executor, SKernel semanticKernel)
    {
        _executor = executor;
        _semanticKernel = semanticKernel;
        var options = new FunctionCallingStepwisePlannerOptions
        {
            MaxIterations = 5,
            ExecutionSettings = new OpenAIPromptExecutionSettings { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions }
        };
        _planner = new FunctionCallingStepwisePlanner(options);
        // _semanticKernel.ImportPluginFromType<WorkbookInteraction>();
    }

    public async Task<string> UpdateNotebook(string notebookPath, string outputPath, string taskDescription)
    {

        var notebook = await InteractiveDocument.LoadAsync(new FileInfo(notebookPath));
        var resultDocument = await _executor.RunNotebookAsync(notebook);

        await File.WriteAllTextAsync(outputPath, resultDocument.ToJupyterJson());
        var outputContent = await File.ReadAllTextAsync(outputPath);

        var result = await _planner.ExecuteAsync(_semanticKernel, "Analyze the output and suggest updates", new Microsoft.SemanticKernel.ChatCompletion.ChatHistory(outputContent));
        return result.FinalAnswer;
    }
}


### 5. Exécution et Mise à Jour Itérative

In [47]:
using Microsoft.DotNet.Interactive.CSharp;

var compositeKernel = new CompositeKernel
{
    new CSharpKernel()
};
var executor = new NotebookExecutor(compositeKernel);
var updater = new NotebookUpdater(executor, semanticKernel);

var response = await updater.UpdateNotebook(notebookPath, notebookOutputPath, taskDescription);

Console.WriteLine("Résultat de l'exécution du notebook :\n" + response);


Error: Command cancelled.

### Conclusion